<a href="https://colab.research.google.com/github/JonathanLehner/Colab-collection/blob/master/train_multi_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# connect to google drive, need to copy auth code from new page
from google.colab import drive
# gdrive not working somehow
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# define variables
import os
repository_url = 'https://github.com/JonathanLehner/Colab-collection'
repository_name = os.path.basename(repository_url)
loader_path = './products.py'
training_data_url = 'https://github.com/JonathanLehner/Colab-collection/releases/download/0.1/labeled_training_images.zip'
training_filename = os.path.basename(training_data_url)
training_filename_wo_ext = os.path.splitext(training_filename)[0]

training_mode = "last" # coco or last
last_model_filename = "mask_rcnn_products_0001.h5"
epochs = 90

In [3]:
print(repository_name)

Colab-collection


In [4]:
 # clone Mask-RCNN repo
%cd /content/
import os
from os.path import exists, join, basename

project_name = "Mask_RCNN"
if not exists(project_name):
  # clone and install
  !git clone -q https://github.com/matterport/Mask_RCNN.git
  !cd $project_name && pip install -q -r requirements.txt
  
import sys
sys.path.append(project_name)

/content
     |████████████████████████████████| 204kB 2.9MB/s 


In [5]:
%cd /content/Mask_RCNN
import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt

# Root directory of the project
ROOT_DIR = os.path.abspath("./")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

/content/Mask_RCNN


Using TensorFlow backend.


... done downloading pretrained model!


In [6]:
%cd /content/Mask_RCNN

/content/Mask_RCNN


In [7]:
#clean up in case of repetitions
#!rm huerlimann3.zip
#!rm -r huerlimann3
!rm -r __MACOSX
!rm -r /content/Mask_RCNN/datasets

rm: cannot remove '__MACOSX': No such file or directory
rm: cannot remove '/content/Mask_RCNN/datasets': No such file or directory


In [8]:
#load and unzip training data
os.system('wget %s'%training_data_url)
os.system('unzip %s'%training_filename)


0

In [9]:
!mkdir /content/Mask_RCNN/datasets
os.system('mv /content/Mask_RCNN/{0} /content/Mask_RCNN/datasets/{1}'.format(training_filename_wo_ext,training_filename_wo_ext))

0

In [0]:
#!rm -r /content/Mask_RCNN/samples/Colab-collection
#print('git clone {0}'.format(repository_url))
#!git clone https://github.com/JonathanLehner/Colab-collection

In [11]:
# load the training script
%cd /content/Mask_RCNN/samples/
os.system('rm -r {0}'.format(repository_name))

print('git clone {0}'.format(repository_url))
os.system('git clone {0}'.format(repository_url))

/content/Mask_RCNN/samples
git clone https://github.com/JonathanLehner/Colab-collection


0

In [12]:
# how to import products.py file 
'''os.system('/content/Mask_RCNN/samples/{0}'.format(repository_name))
products = __import__("products")
print(products)
config = products.ProductConfig()
PRODUCT_DIR = os.path.join(ROOT_DIR, "datasets/"+repository_name)'''



'os.system(\'/content/Mask_RCNN/samples/{0}\'.format(repository_name))\nproducts = __import__("products")\nprint(products)\nconfig = products.ProductConfig()\nPRODUCT_DIR = os.path.join(ROOT_DIR, "datasets/"+repository_name)'

In [13]:
#until then use https://www.dlology.com/blog/quick-guide-to-run-tensorboard-in-google-colab/
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2019-05-04 19:05:47--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.21.103.149, 52.203.53.176, 34.206.130.40, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.21.103.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14991793 (14M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  14.30M  86.3MB/s    in 0.2s    

2019-05-04 19:05:47 (86.3 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [14991793/14991793]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [14]:
#download the model file from drive
!mkdir /content/Mask_RCNN/logs
import datetime
now = datetime.datetime.now()
LOG_DIR = 'products{:%Y%m%dT%H%M}'.format(now)
os.system('mkdir /content/Mask_RCNN/logs/{0}'.format(LOG_DIR))
print('cp "/content/gdrive/My Drive/{1}" /content/Mask_RCNN/logs/{0}/{1}'.format(LOG_DIR, last_model_filename))
os.system('cp "/content/gdrive/My Drive/{1}" /content/Mask_RCNN/logs/{0}/{1}'.format(LOG_DIR, last_model_filename))

cp "/content/gdrive/My Drive/mask_rcnn_products_0001.h5" /content/Mask_RCNN/logs/products20190504T1905/mask_rcnn_products_0001.h5


0

In [0]:
#!cp "/content/gdrive/My Drive/mask_rcnn_products_0001.h5" /content/Mask_RCNN/logs/20190504T1850/mask_rcnn_products_0001.h5


In [15]:
# this code crashes keras, but will work in the future (hopefully)
'''
# activate tensorboard
# https://github.com/tensorflow/tensorboard/blob/master/docs/r2/tensorboard_in_notebooks.ipynb

!pip install -q tf-nightly-2.0-preview

# Load the TensorBoard notebook extension
%load_ext tensorboard

import tensorflow as tf
import datetime, os

%tensorboard --logdir logs '''



import datetime
now = datetime.datetime.now()
LOG_DIR = '/content/Mask_RCNN/logs/products{:%Y%m%dT%H%M}'.format(now)
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

get_ipython().system_raw('./ngrok http 6006 &')

! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"


http://6b009842.ngrok.io


In [0]:
#imgaug is included in MaskRCNN
import os
# call script with subprocess
# train
os.chdir('/content/Mask_RCNN/samples/{0}'.format(repository_name))
# !pwd
# use weights=last to continue training model
command_string = 'python3 {0} train --dataset=/content/Mask_RCNN/datasets/{1} --weights={2} --epochs={3} --logs=/content/Mask_RCNN/logs'.format(loader_path, training_filename_wo_ext, training_mode, epochs)

import subprocess
import shlex
args = shlex.split(command_string)
print(args)
process = subprocess.Popen(args, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, bufsize=1)
#inspiration from https://www.endpoint.com/blog/2015/01/28/getting-realtime-output-using-python
while True:
    output = process.stdout.readline()
    if process.poll() is not None:
        break
    if output != b'':
        print(output)
    rc = process.poll()



['python3', './products.py', 'train', '--dataset=/content/Mask_RCNN/datasets/labeled_training_images', '--weights=last', '--epochs=90', '--logs=/content/Mask_RCNN/logs']
b'Using TensorFlow backend.\n'
b'WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/op_def_library.py:263: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.\n'
b'Instructions for updating:\n'
b'Colocations handled automatically by placer.\n'
b'2019-05-04 19:06:07.253036: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz\n'
b'2019-05-04 19:06:07.253528: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x1844a680 executing computations on platform Host. Devices:\n'
b'2019-05-04 19:06:07.253559: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>\n'
b'2019-05-04 19:06:07.500471: I tensorflow/stream_executor/cuda/cuda_gpu_exec

In [0]:
#need to kill aborted process to release GPU memory again
!ps
#!kill 501

#this doesn't work: process.terminate()
#neither does this: get_ipython().system_raw('kill -9 $(sudo lsof -t -i:%d)' % 6006)

In [0]:
# find newest model file
# we save a model in drive to allow easier use of tensorboard
!ls /content/Mask_RCNN/logs/
from os import walk

logs = []
for (dirpath, dirnames, filenames) in walk("/content/Mask_RCNN/logs/"):
    logs.extend(dirnames)
    break

print(logs)
logs.sort(reverse=True) # get the most recent log
last_log = logs[0]

files = []
for (dirpath, dirnames, filenames) in walk("/content/Mask_RCNN/logs/"+last_log):
    files.extend(filenames)
    break
    
checkpoints = sorted(filter(lambda f: f.startswith("mask_rcnn"), files), reverse=True)
print(checkpoints)

last_model = checkpoints[0]


In [0]:
# create zipfile of all logs
# https://stackoverflow.com/questions/1855095/how-to-create-a-zip-archive-of-a-directory
# import shutil
# shutil.make_archive('/content/logs.zip', 'zip', '/content/Mask_RCNN/logs/')


In [0]:
#download the model file
from google.colab import drive
#gdrive not working somehow
drive.mount('/content/gdrive')
print('cp /content/Mask_RCNN/logs/{0}/{1} "/content/gdrive/My Drive/{1}"'.format(last_log, last_model, last_model))
os.system('cp /content/Mask_RCNN/logs/{0}/{1} "/content/gdrive/My Drive/{1}"'.format(last_log, last_model, last_model))

In [0]:
# copy the line from the top to do the actual transfer --> seems to work better like this
!cp /content/Mask_RCNN/logs/products20190504T1822/mask_rcnn_products_0001.h5 "/content/gdrive/My Drive/mask_rcnn_products_0001.h5"

In [0]:
#somehow not working for big files, only directly with cp works
os.system('cp /content/Mask_RCNN/logs/{0}/{1} /content/gdrive/My Drive/{1}'.format(last_log, last_model, last_model))

In [0]:
# https://stackoverflow.com/questions/49428332/how-to-download-large-files-like-weights-of-a-model-from-colaboratory
# from google.colab import files
# files.download('/content/Mask_RCNN/logs/{0}/{1}'.format(last_log, last_model))

In [0]:
#copy needs ""
#!cp /content/Mask_RCNN/logs/products20190430T2145/mask_rcnn_products_0030.h5 "/content/gdrive/My Drive/"
